# Lancer un processus externe

Python est parfaitement capable de lancer un programme externe, en particulier un programme en ligne de commande comme EnergyPlus. En natif, cela passe par la librairie builtin `subprocess`. L'utilisation de cette librairie peut être un peu délicate, et nous allons aussi étudier l'utilisation de `plumbum`, une librairie tierce partie facilitant la manipulation de commandes systèmes.

**Etant donnée la différence d'OS, il sera impossible d'executer exactement les même commandes depuis linux ou windows**

## Avec subprocess

In [1]:
import subprocess
from path import Path

In [2]:
eplus_bin = Path('/home/nicolas/.local/share/energy_plus_wrapper/EnergyPlus-8-7-0/energyplus')
idf_file = Path("../../data/eplus_data/in_8-7-0.idf")
epw_file = Path("../../data/eplus_data/in.epw")
out_dir = Path("./output_eplus/")

la fonction `run` permet de lancer le process et d'en attendre le résultat. avec l'argument `capture_output`, on récupère les messages de sorties d'erreurs (resp. stdout et stderr).

Il faut fournir une liste d'arguments, le premier étant la commande à lancer, suivi de tous ses arguments.

Attention, les output sont en format binaire : il faut utiliser la méthode `decode` pour les afficher correctement.

In [3]:
help_eplus = subprocess.run([eplus_bin, "--help"], capture_output=True)
print(help_eplus.stdout.decode())

EnergyPlus, Version 8.7.0-78a111df4a
Usage: energyplus [options] [input-file]
Options:
  -a, --annual                 Force annual simulation
  -D, --design-day             Force design-day-only simulation
  -d, --output-directory ARG   Output directory path (default: current
                               directory)
  -h, --help                   Display help information
  -i, --idd ARG                Input data dictionary path (default: Energy+.idd
                               in executable directory)
  -m, --epmacro                Run EPMacro prior to simulation
  -p, --output-prefix ARG      Prefix for output file names (default: eplus)
  -r, --readvars               Run ReadVarsESO after simulation
  -s, --output-suffix ARG      Suffix style for output file names (default: L)
                                  L: Legacy (e.g., eplustbl.csv)
                                  C: Capital (e.g., eplusTable.csv)
                                  D: Dash (e.g., eplus-table.csv)
  -v, -

On a une meilleur idée de comment utiliser la commande :

```bash
../path/to/eplus_bin -w ../../data/eplus_data/in.epw -d output_eplus ../../data/eplus_data/in_8-7-0.idf
```

devrait fonctionner.

In [4]:
process = subprocess.run([eplus_bin, "-w", epw_file, "-d", out_dir, idf_file],
                         capture_output=True)
process

CompletedProcess(args=[Path('/home/nicolas/.local/share/energy_plus_wrapper/EnergyPlus-8-7-0/energyplus'), '-w', Path('../../data/eplus_data/in.epw'), '-d', Path('./output_eplus/'), Path('../../data/eplus_data/in_8-7-0.idf')], returncode=0, stdout=b'EnergyPlus Starting\nEnergyPlus, Version 8.7.0-78a111df4a, YMD=2020.02.28 14:58\nProcessing Data Dictionary\nProcessing Input File\nInitializing Response Factors\nCalculating CTFs for "STEEL FRAME NON-RES EXT WALL", Construction # 2\nCalculating CTFs for "IEAD NON-RES ROOF", Construction # 3\nCalculating CTFs for "EXT-SLAB", Construction # 4\nCalculating CTFs for "INT-WALLS", Construction # 5\nCalculating CTFs for "DROPCEILING", Construction # 7\nCalculating CTFs for "INTERIORFURNISHINGS", Construction # 9\nInitializing Window Optical Properties\nInitializing Solar Calculations\nAllocate Solar Module Arrays\nInitializing Zone Report Variables\nInitializing Surface (Shading) Report Variables\nDetermining Shadowing Combinations\nComputing Win

On a fait tourner EnergyPlus depuis Python !

On peut déjà remarquer que le dossier de sortie s'est bien remplie.

In [6]:
print(process.stdout.decode())

EnergyPlus Starting
EnergyPlus, Version 8.7.0-78a111df4a, YMD=2020.02.28 14:58
Processing Data Dictionary
Processing Input File
Initializing Response Factors
Calculating CTFs for "STEEL FRAME NON-RES EXT WALL", Construction # 2
Calculating CTFs for "IEAD NON-RES ROOF", Construction # 3
Calculating CTFs for "EXT-SLAB", Construction # 4
Calculating CTFs for "INT-WALLS", Construction # 5
Calculating CTFs for "DROPCEILING", Construction # 7
Calculating CTFs for "INTERIORFURNISHINGS", Construction # 9
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone Report Variables
Initializing Surface (Shading) Report Variables
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializing Solar Calculations
Initializing Surfaces
Initializing Outdoor environment for Surfaces
Setting up Surface Reporting Variables
Initializing Temperature and Flux Histories
Initializing Window Shading
Computing I

In [5]:
print(*out_dir.files(), sep="\n")

./output_eplus/eplusout.dxf
./output_eplus/eplusout.rdd
./output_eplus/eplusout.eso
./output_eplus/eplusout.eio
./output_eplus/eplusout.mdd
./output_eplus/eplusout.bnd
./output_eplus/eplusout.mtd
./output_eplus/eplusout.sql
./output_eplus/sqlite.err
./output_eplus/eplusout.mtr
./output_eplus/eplusout.shd
./output_eplus/eplustbl.htm
./output_eplus/eplusssz.csv
./output_eplus/epluszsz.csv
./output_eplus/eplusout.err
./output_eplus/eplusout.audit
./output_eplus/eplusout.end


Energy Plus est très verbeux et génère beaucoup de fichiers intermédiaires. Cela peut poser soucis pour faire tourner plusieurs instances en même temps.

On va voir plus tard comment gérer tout ces fichiers.

## Avec plumbum

Plumbum permet de manipuler plus aisément les commandes externes. Il permet de transformer une commande externe en fonction python, et renverra directement l'output sous forme de chaine de caractère.

```pip install plumbum```

In [7]:
import plumbum

In [9]:
eplus = plumbum.local[eplus_bin]

In [10]:
output_eplus = eplus("-w", epw_file, "-d", out_dir, idf_file)

In [11]:
print(output_eplus)

EnergyPlus Starting
EnergyPlus, Version 8.7.0-78a111df4a, YMD=2020.02.28 15:01
Processing Data Dictionary
Processing Input File
Initializing Response Factors
Calculating CTFs for "STEEL FRAME NON-RES EXT WALL", Construction # 2
Calculating CTFs for "IEAD NON-RES ROOF", Construction # 3
Calculating CTFs for "EXT-SLAB", Construction # 4
Calculating CTFs for "INT-WALLS", Construction # 5
Calculating CTFs for "DROPCEILING", Construction # 7
Calculating CTFs for "INTERIORFURNISHINGS", Construction # 9
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone Report Variables
Initializing Surface (Shading) Report Variables
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializing Solar Calculations
Initializing Surfaces
Initializing Outdoor environment for Surfaces
Setting up Surface Reporting Variables
Initializing Temperature and Flux Histories
Initializing Window Shading
Computing I

Il est possible de récupérer la sortie en temps réel, mais au pris de manipulation plus complexes. On utilisera alors `subprocess.Popen` qui renverra un objet pendant que la commande tourne en arrière plan. De cette façon, il est possible de scruter la sortie de la commande pendant qu'elle tourne.

process.stdout.readline()

In [13]:
def run_with_rt_output(args):
    # On crée un processus avec Popen. Celui ci va tourner en arrière plan.
    process = subprocess.Popen(args, stdout=subprocess.PIPE)
    # On va transformer la méthode `readline` de la sortie standard en iterable.
    # le second paramètre est une *sentinelle* : la fonction est appelé jusqu'à
    # ce qu'elle retourne la valeur sentinelle.
    for output in iter(process.stdout.readline, b""):
        print(output.decode().strip())
    # on retourne le code de sortie.
    return process.poll()

In [14]:
run_with_rt_output([eplus_bin, "-w", epw_file, "-d", out_dir, idf_file])

EnergyPlus Starting
EnergyPlus, Version 8.7.0-78a111df4a, YMD=2020.02.28 15:03
Processing Data Dictionary
Processing Input File
Initializing Response Factors
Calculating CTFs for "STEEL FRAME NON-RES EXT WALL", Construction # 2
Calculating CTFs for "IEAD NON-RES ROOF", Construction # 3
Calculating CTFs for "EXT-SLAB", Construction # 4
Calculating CTFs for "INT-WALLS", Construction # 5
Calculating CTFs for "DROPCEILING", Construction # 7
Calculating CTFs for "INTERIORFURNISHINGS", Construction # 9
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone Report Variables
Initializing Surface (Shading) Report Variables
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializing Solar Calculations
Initializing Surfaces
Initializing Outdoor environment for Surfaces
Setting up Surface Reporting Variables
Initializing Temperature and Flux Histories
Initializing Window Shading
Computing I